In [1]:
import pandas as pd

# Examine the seed datasets
ISIS_seed = pd.read_csv('Seed_MIWS/Seed_Dataset/ISIS_Seed_Complete.csv')
WS_seed = pd.read_csv('Seed_MIWS/Seed_Dataset/WS_Seed_Complete.csv')

display(ISIS_seed.head())
display(WS_seed.head())

,Source,Type of Source,Text,Label,Geographical_Location,Author_Country_Affiliation,Unnamed: 6
0,"Chatfield et.al ""Tweeting propaganda, radicali...",Research Article,Coalition planes massacred these children in a...,Propaganda,Iraq,USA,NaN
1,"Chatfield et.al ""Tweeting propaganda, radicali...",Research Article,these PKK fellas are exceptional liars.after t...,Propaganda,Iraq,USA,NaN
2,"Chatfield et.al ""Tweeting propaganda, radicali...",Research Article,This is so awesome. US airstrikes also by mist...,Propaganda,Iraq,USA,NaN
3,"Chatfield et.al ""Tweeting propaganda, radicali...",Research Article,RT @ImtiyazAzhar: Support &amp; love for #Isla...,Propaganda,India,USA,NaN
4,"Chatfield et.al ""Tweeting propaganda, radicali...",Research Article,Ask the Americans how they liked fighting\nJTJ...,Propaganda,USA,USA,NaN


,Source,Type_of_Source,Text,Ideology,Label,Geographical_Location,Author_Country_Affiliation,Unnamed: 7
0,"Ray and Marsh ""Recruitment by extremist groups...",Research Article,This is a deliberate choice of words. As we st...,White Supremacist,Propaganda,-,USA,NaN
1,"Ray and Marsh ""Recruitment by extremist groups...",Research Article,Most victims of race crime - about 90 per cent...,White Supremacist,Propaganda,-,USA,NaN
2,"Ray and Marsh ""Recruitment by extremist groups...",Research Article,WE BELIEVE that the Cananite Jew is the natura...,White Supremacist,Radicalization,-,USA,NaN
3,"Ray and Marsh ""Recruitment by extremist groups...",Research Article,"The culture of a race, free of alien influence...",White Supremacist,Radicalization,-,USA,NaN
4,"Ray and Marsh ""Recruitment by extremist groups...",Research Article,Influential organizations and much of the west...,White Supremacist,Propaganda,"Switzerland, Germany",USA,NaN


In [2]:
import numpy as np

"""
Given a path to a .csv file, this function will read the file and format it as a dicionary with the following structure:
    
    {
        'Inputs': [message1, message2, ...],
        'Labels': [label1, label2, ...]
    }
    
    where 'Inputs' is a list of lists of message dictionaries that conform to the input to OpenAI's Chat Completion API,
    and 'Labels' is a list of labels for each message. Each label is a string that represents the type of extremist content of the message.

    Each message should have the following format:
    [
        {
            'role': 'system',
            'content': 'system message'
        },
        {
            'role': 'user',
            'content': 'tweet text'
        }
    ]
"""
def format_eval_openai(path, system_message, labels_present=True, limit=None):
    df = pd.read_csv(path, encoding='latin-1')
    inputs = []
    labels = []
    for index, row in df.iterrows():
        if limit and index >= limit:
            break
        tweet_text = row['Text']
        inputs.append([
            {
                'role': 'system',
                'content': system_message
            },
            {
                'role': 'user',
                'content': tweet_text
            }
        ])
        if labels_present:
            labels.append(row['Label'])
        else:
            labels.append("None")
    return {
        'Inputs': inputs,
        'Labels': labels
    }

In [3]:
"""
Given a path to a .csv file, this function will read the file and format it as a dicionary with the following structure:
        
        {
            'Inputs': [request1, request2, ...]),
            'Labels': [label1, label2, ...]
        }
        
        where 'Inputs' is a list of request dictionaries that conform to the input to Perspective's comments().analyze API,
        and 'Labels' is a list of labels for each message. Each label is a string that represents the type of extremist content of the message.
    
        Each request will have the following format:
        {
            'comment': { 'text': 'tweet text' },
            'requestedAttributes': metrics
        }

Args:
    path: The path to the .csv file
    metrics: A dictionary of the metrics to request from the Perspective API (example: {'TOXICITY': {}, 'SEVERE_TOXICITY': {}})
"""
def format_eval_perspective(path, metrics, labels_present=True, limit=None):
    df = pd.read_csv(path, encoding='latin-1')
    inputs = []
    labels = []
    for index, row in df.iterrows():
        if limit and index >= limit:
            break
        tweet_text = row['Text']
        inputs.append({
            'comment': { 'text': tweet_text },
            'requestedAttributes': metrics
        })
        if labels_present:
            labels.append(row['Label'])
        else:
            labels.append("None")
    return {
        'Inputs': inputs,
        'Labels': labels
    }

In [4]:
system_message = "You are on a social media platform's moderation team. "
system_message += "Your task is to decide whether the following tweet contains extremist content or not.\n"
system_message += "Extremist content includes messages that are considered to be propoganda, radicalization, or recruitment. "
system_message += "Please label the tweet as 'Propoganda', 'Radicalization', 'Recruitment', or 'None'.\n"
system_message += "Your output must be in JSON format and with the following structure:\n{\"Label\": \"Propoganda\" or \"Radicalization\" or \"Recruitment\" or \"None\"}"

print(system_message)

ISIS_dict_openai = format_eval_openai('Seed_MIWS/Seed_Dataset/ISIS_Seed_Complete.csv', system_message)
WS_dict_openai = format_eval_openai('Seed_MIWS/Seed_Dataset/WS_Seed_Complete.csv', system_message)
tweets_dict_openai = format_eval_openai("tweets.csv", system_message, labels_present=False, limit=1000)

metrics = {'IDENTITY_ATTACK': {}, 'SEVERE_TOXICITY': {}, 'THREAT': {}}
ISIS_dict_perspective = format_eval_perspective('Seed_MIWS/Seed_Dataset/ISIS_Seed_Complete.csv', metrics)
WS_dict_perspective = format_eval_perspective('Seed_MIWS/Seed_Dataset/WS_Seed_Complete.csv', metrics)
tweets_dict_perspective = format_eval_openai("tweets.csv", metrics, labels_present=False, limit=1000)

You are on a social media platform's moderation team. Your task is to decide whether the following tweet contains extremist content or not.
Extremist content includes messages that are considered to be propoganda, radicalization, or recruitment. Please label the tweet as 'Propoganda', 'Radicalization', 'Recruitment', or 'None'.
Your output must be in JSON format and with the following structure:
{"Label": "Propoganda" or "Radicalization" or "Recruitment" or "None"}


In [5]:
combined_dict_openai = {
    'Inputs': ISIS_dict_openai['Inputs'] + WS_dict_openai['Inputs'] + tweets_dict_openai['Inputs'],
    'Labels': ISIS_dict_openai['Labels'] + WS_dict_openai['Labels'] + tweets_dict_openai['Labels']
}

combined_dict_perspective = {
    'Inputs': ISIS_dict_perspective['Inputs'] + WS_dict_perspective['Inputs'] + tweets_dict_perspective['Inputs'],
    'Labels': ISIS_dict_perspective['Labels'] + WS_dict_perspective['Labels'] + tweets_dict_perspective['Labels']
}

In [6]:
print("Combined openAI dictionary:")
print(f"Inputs shape: {len(combined_dict_openai['Inputs'])}")
print(f"Labels shape: {len(combined_dict_openai['Labels'])}")

print("")

print("Combined Perspective dictionary:")
print(f"Inputs shape: {len(combined_dict_perspective['Inputs'])}")
print(f"Labels shape: {len(combined_dict_perspective['Labels'])}")

Combined openAI dictionary:
Inputs shape: 1398
Labels shape: 1398

Combined Perspective dictionary:
Inputs shape: 1398
Labels shape: 1398


In [7]:
# Save the combined dictionary to a new .json file at the specified path
import json
json_file_openai = json.dumps(combined_dict_openai, indent=4)
json_file_perspective = json.dumps(combined_dict_perspective, indent=4)

with open("eval_data_openai.json", "w") as file:
    file.write(json_file_openai)

with open("eval_data_perspective.json", "w") as file:
    file.write(json_file_perspective)

In [8]:
print(combined_dict_openai['Inputs'][0])

[{'role': 'system', 'content': 'You are on a social media platform\'s moderation team. Your task is to decide whether the following tweet contains extremist content or not.\nExtremist content includes messages that are considered to be propoganda, radicalization, or recruitment. Please label the tweet as \'Propoganda\', \'Radicalization\', \'Recruitment\', or \'None\'.\nYour output must be in JSON format and with the following structure:\n{"Label": "Propoganda" or "Radicalization" or "Recruitment" or "None"}'}, {'role': 'user', 'content': 'Coalition planes massacred these children in airstrikes\nOn #Hit ,#Anbar\nhttp://t.co/yCsEgkwDY6#Iraq'}]


In [9]:
print(combined_dict_perspective['Inputs'][0])

{'comment': {'text': 'Coalition planes massacred these children in airstrikes\nOn #Hit ,#Anbar\nhttp://t.co/yCsEgkwDY6#Iraq'}, 'requestedAttributes': {'IDENTITY_ATTACK': {}, 'SEVERE_TOXICITY': {}, 'THREAT': {}}}


In [10]:
gemini_sys_message = '''You are on a social media platform's moderation team. Your task is to decide whether the tweets provided to you contain extremist content or not.
Extremist content includes messages that are considered to be propoganda, radicalization, or recruitment. Please label each tweet as 'Propoganda', 'Radicalization', 'Recruitment', or 'None'.
The format of the messages supplied to you will be as follows: "Please label the following tweet as 'Propoganda', 'Radicalization', 'Recruitment', or 'None': [TWEET], with id of ID", where TWEET will be replaced by each tweet's text content, and ID is the tweet's id.
For each tweet, your output must be in JSON format and with the following structure:
{"ID": ID, "Label": "Propoganda" or "Radicalization" or "Recruitment" or "None"}, where ID is the tweet's ID that was provided to you in the message.'''
print(gemini_sys_message)

You are on a social media platform's moderation team. Your task is to decide whether the tweets provided to you contain extremist content or not.
Extremist content includes messages that are considered to be propoganda, radicalization, or recruitment. Please label each tweet as 'Propoganda', 'Radicalization', 'Recruitment', or 'None'.
The format of the messages supplied to you will be as follows: "Please label the following tweet as 'Propoganda', 'Radicalization', 'Recruitment', or 'None': [TWEET], with id of ID", where TWEET will be replaced by each tweet's text content, and ID is the tweet's id.
For each tweet, your output must be in JSON format and with the following structure:
{"ID": ID, "Label": "Propoganda" or "Radicalization" or "Recruitment" or "None"}, where ID is the tweet's ID that was provided to you in the message.


In [11]:
import json
 
with open("../DiscordBot/tokens.json", 'r') as file:
    tokens = json.load(file)


## OLD version - this will error out due to hitting safety filters. See further below for working Gemini implementation


In [12]:
!pip install vertexai

In [13]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=OzpVgwwBIOmCVtQhzBEtGustkIaHnp&access_type=offline&code_challenge=Tb5VyM87OvKtUUwyTkl1v-exRGulHy0eGr-xskQKCgA&code_challenge_method=S256


You are now logged in as [starishi@gmail.com].
Your current project is [cs152-424817].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [14]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=jxLsq4oL5ErcfvMam7stxBMcIVA0QX&access_type=offline&code_challenge=MXxtbTG2vC_blOpCmTKjzdaWlsXi01nN0-Yp-3xOqgk&code_challenge_method=S256


Credentials saved to file: [/Users/rishisreekanth/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "cs152-424817" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning t

In [15]:
import vertexai

from vertexai.generative_models import GenerativeModel, ChatSession

project_id = tokens['gcp_project_id']

vertexai.init(project=project_id, location="us-central1")

model = GenerativeModel(
    "gemini-1.5-flash"
)

chat = model.start_chat(response_validation=False)

def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response)

prompt = '''
    I need you to review the following content to determine if it contains harmful language. The content should not be repeated or generated. Please analyze the text and provide feedback on whether it includes any of the following categories of extremist language: 'Propaganda', 'Radicalization', 'Recruitment', or 'None'.

Content to review:
[Coalition planes massacred these children in airstrikes\nOn #Hit ,#Anbar\nhttp://t.co/yCsEgkwDY6#Iraq]

Please respond with a summary indicating whether any extremist language is present and categorize it accordingly.
'''
print(get_chat_response(chat, prompt))

# prompt = "What are all the colors in a rainbow?"
# print(get_chat_response(chat, prompt))

# prompt = "Why does it appear when it rains?"
# print(get_chat_response(chat, prompt))

ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "finish_reason": "SAFETY",
  "safety_ratings": [
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "probability": "LOW",
      "probability_score": 0.5176623,
      "severity": "HARM_SEVERITY_MEDIUM",
      "severity_score": 0.4387551
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "probability": "MEDIUM",
      "blocked": true,
      "probability_score": 0.7816642,
      "severity": "HARM_SEVERITY_HIGH",
      "severity_score": 0.7218413
    },
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "probability": "LOW",
      "probability_score": 0.61503005,
      "severity": "HARM_SEVERITY_MEDIUM",
      "severity_score": 0.57080984
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "probability": "NEGLIGIBLE",
     

## WORKING version 


In [16]:
!pip install -q -U google-generativeai

In [17]:
sample_prompt = '''You are on a social media platform's moderation team. I need you to review the following content as if it were a tweet to determine if it contains harmful and/or extremist language. The content should not be repeated or generated. Please analyze the text and provide feedback on whether it includes any of the following categories of extremist language, and choose the one that fits best: 'Propaganda', 'Radicalization', 'Recruitment', or 'None'.

Content to review:
[Coalition planes massacred these children in airstrikes\nOn #Hit ,#Anbar\nhttp://t.co/yCsEgkwDY6#Iraq]

Please respond with a summary indicating whether any extremist language is present and categorize it in only one of the provided categories accordingly.
Please give your response as valid JSON in the following format: {"Label": <INSERT YOUR CATEGORIZATION HERE ("Propaganda" or "Radicalization" or "Recruitment" or "None")>, "Reason": <INSERT YOUR REASON HERE>}.
'''

In [33]:
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

genai.configure(api_key=tokens["gemini_google_ai_studio"])
model = genai.GenerativeModel(model_name='gemini-1.5-flash')
def review_prompt(prompt):
    response = model.generate_content(
        prompt,
        safety_settings={
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
        }
    )
    try:
        res = response.text
        res_json = json.loads(response.text[7:][:-5])
        return res_json
    except:
        # If the response doesn't contain text, check if the prompt was blocked.
        print(response.prompt_feedback)
        if len(response.candidates) > 0:
            # Also check the finish reason to see if the response was blocked.
            print(response.candidates[0].finish_reason)
            # If the finish reason was SAFETY, the safety ratings have more details.
            print(response.candidates[0].safety_ratings)
        return {"error": str(response.prompt_feedback)}
        



In [34]:
def format_eval_gemini(path, system_message, labels_present=True, limit=None):
    df = pd.read_csv(path, encoding='latin-1')
    inputs = []
    labels = []
    for index, row in df.iterrows():
        if limit and index >= limit:
            break
        tweet_text = row['Text']
        prompt = "You are on a social media platform's moderation team. I need you to review the following content as if it were a tweet to determine if it contains harmful and/or extremist language. The content should not be repeated or generated. Please analyze the text and provide feedback on whether it includes any of the following categories of extremist language, and choose the one that fits best: 'Propaganda', 'Radicalization', 'Recruitment', or 'None'.\n\n"
        prompt += "Content to review:\n"
        prompt += f"[{tweet_text}]\n\n"
        prompt += "Please respond with a summary indicating whether any extremist language is present and categorize it in only one of the provided categories accordingly.\n"
        prompt += "Please give your response as valid JSON in the following format: {\"Label\": <INSERT YOUR CATEGORIZATION HERE (\"Propaganda\" or \"Radicalization\" or \"Recruitment\" or \"None\")>, \"Reason\": <INSERT YOUR REASON HERE>}."
        inputs.append(
            {
                'content': prompt
            }
        )
        if labels_present:
            labels.append(row['Label'])
        else:
            labels.append("None")
    return {
        'Inputs': inputs,
        'Labels': labels
    }

In [35]:
ISIS_dict_gemini = format_eval_gemini('Seed_MIWS/Seed_Dataset/ISIS_Seed_Complete.csv', metrics)
WS_dict_gemini = format_eval_gemini('Seed_MIWS/Seed_Dataset/WS_Seed_Complete.csv', metrics)
tweets_dict_gemini = format_eval_gemini("tweets.csv", gemini_sys_message, labels_present=False, limit=1000)

In [36]:
combined_dict_gemini = {
    'Inputs': ISIS_dict_gemini['Inputs'] + WS_dict_gemini['Inputs'] + tweets_dict_gemini['Inputs'],
    'Labels': ISIS_dict_gemini['Labels'] + WS_dict_gemini['Labels'] + tweets_dict_gemini['Labels']
}


In [37]:
print("Combined Gemini dictionary:")
print(f"Inputs shape: {len(combined_dict_gemini['Inputs'])}")
print(f"Labels shape: {len(combined_dict_gemini['Labels'])}")

Combined Gemini dictionary:
Inputs shape: 1398
Labels shape: 1398


In [38]:
json_file_gemini = json.dumps(combined_dict_gemini, indent=4)

with open("eval_data_gemini.json", "w") as file:
    file.write(json_file_gemini)

In [39]:
predicted = []
for i in range(len(combined_dict_gemini['Inputs'])):
    predicted.append(review_prompt(combined_dict_gemini['Inputs'][i]['content']))
    print(f"finished {i} out of {len(combined_dict_gemini['Inputs'])}")

finished 0 out of 1398
finished 1 out of 1398
finished 2 out of 1398
finished 3 out of 1398
finished 4 out of 1398
finished 5 out of 1398
finished 6 out of 1398
finished 7 out of 1398
finished 8 out of 1398
finished 9 out of 1398
finished 10 out of 1398
finished 11 out of 1398
finished 12 out of 1398
finished 13 out of 1398
finished 14 out of 1398
finished 15 out of 1398
finished 16 out of 1398
finished 17 out of 1398
finished 18 out of 1398
finished 19 out of 1398
finished 20 out of 1398
finished 21 out of 1398
finished 22 out of 1398
finished 23 out of 1398
finished 24 out of 1398
finished 25 out of 1398
finished 26 out of 1398
finished 27 out of 1398
finished 28 out of 1398
finished 29 out of 1398
finished 30 out of 1398
finished 31 out of 1398
finished 32 out of 1398
finished 33 out of 1398
finished 34 out of 1398
finished 35 out of 1398
finished 36 out of 1398
finished 37 out of 1398
finished 38 out of 1398
finished 39 out of 1398
finished 40 out of 1398
finished 41 out of 1398
fi

finished 332 out of 1398
finished 333 out of 1398
finished 334 out of 1398
finished 335 out of 1398
finished 336 out of 1398
finished 337 out of 1398
finished 338 out of 1398
finished 339 out of 1398
finished 340 out of 1398
finished 341 out of 1398
finished 342 out of 1398
finished 343 out of 1398
finished 344 out of 1398
finished 345 out of 1398
finished 346 out of 1398
finished 347 out of 1398
finished 348 out of 1398
finished 349 out of 1398
finished 350 out of 1398
finished 351 out of 1398
finished 352 out of 1398
finished 353 out of 1398
finished 354 out of 1398
finished 355 out of 1398
finished 356 out of 1398
finished 357 out of 1398
finished 358 out of 1398
finished 359 out of 1398
finished 360 out of 1398
finished 361 out of 1398
finished 362 out of 1398
finished 363 out of 1398
finished 364 out of 1398
finished 365 out of 1398
finished 366 out of 1398
finished 367 out of 1398
finished 368 out of 1398
finished 369 out of 1398
finished 370 out of 1398
finished 371 out of 1398


finished 660 out of 1398
finished 661 out of 1398
finished 662 out of 1398
finished 663 out of 1398
finished 664 out of 1398
finished 665 out of 1398
finished 666 out of 1398
finished 667 out of 1398
finished 668 out of 1398
finished 669 out of 1398
finished 670 out of 1398
finished 671 out of 1398
finished 672 out of 1398
finished 673 out of 1398
finished 674 out of 1398
finished 675 out of 1398
finished 676 out of 1398
finished 677 out of 1398
finished 678 out of 1398
finished 679 out of 1398
finished 680 out of 1398
finished 681 out of 1398
finished 682 out of 1398
finished 683 out of 1398
finished 684 out of 1398
finished 685 out of 1398
finished 686 out of 1398
finished 687 out of 1398
finished 688 out of 1398
finished 689 out of 1398
finished 690 out of 1398
finished 691 out of 1398
finished 692 out of 1398
finished 693 out of 1398
finished 694 out of 1398
finished 695 out of 1398
finished 696 out of 1398
finished 697 out of 1398
finished 698 out of 1398
finished 699 out of 1398


finished 988 out of 1398
finished 989 out of 1398
finished 990 out of 1398
finished 991 out of 1398
finished 992 out of 1398
finished 993 out of 1398
finished 994 out of 1398
finished 995 out of 1398
finished 996 out of 1398
finished 997 out of 1398
finished 998 out of 1398

FinishReason.STOP
[category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HARASSMENT
probability: NEGLIGIBLE
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
]
finished 999 out of 1398
finished 1000 out of 1398
finished 1001 out of 1398
finished 1002 out of 1398
finished 1003 out of 1398
finished 1004 out of 1398
finished 1005 out of 1398
finished 1006 out of 1398
finished 1007 out of 1398
finished 1008 out of 1398
finished 1009 out of 1398
finished 1010 out of 1398
finished 1011 out of 1398
finished 1012 out of 1398
finished 1013 out of 1398
finished 1014 out of 1398
finished 1015 out of 1398
finis

finished 1293 out of 1398
finished 1294 out of 1398
finished 1295 out of 1398
finished 1296 out of 1398
finished 1297 out of 1398
finished 1298 out of 1398
finished 1299 out of 1398
finished 1300 out of 1398
finished 1301 out of 1398
finished 1302 out of 1398
finished 1303 out of 1398
finished 1304 out of 1398
finished 1305 out of 1398
finished 1306 out of 1398
finished 1307 out of 1398
finished 1308 out of 1398
finished 1309 out of 1398
finished 1310 out of 1398
finished 1311 out of 1398
finished 1312 out of 1398
finished 1313 out of 1398
finished 1314 out of 1398
finished 1315 out of 1398
finished 1316 out of 1398
finished 1317 out of 1398
finished 1318 out of 1398
finished 1319 out of 1398
finished 1320 out of 1398
finished 1321 out of 1398
finished 1322 out of 1398
finished 1323 out of 1398
finished 1324 out of 1398
finished 1325 out of 1398
finished 1326 out of 1398
finished 1327 out of 1398
finished 1328 out of 1398
finished 1329 out of 1398
finished 1330 out of 1398
finished 133

In [40]:
print(predicted[0]['Label'])

Propaganda


In [41]:
json_file_predictions_gemini = json.dumps({"predictions": predicted}, indent=4)

with open("predictions_gemini.json", "w") as file:
    file.write(json_file_predictions_gemini)

## GEMINI: Confusion Matrix / Performance Metrics

In [42]:
print(combined_dict_gemini['Labels'][-1],predicted[-1]['Label'])

None None


In [43]:
original_total = len(combined_dict_gemini['Inputs'])
n = 0 # number of tweets that were able to be classified without being blocked by safety filters - 
true_pos = []
true_neg = []
false_pos = []
false_neg = []
for i in range(len(combined_dict_gemini['Labels'])):
    if 'Label' not in predicted[i]:
        continue
    if predicted[i]['Label'] == 'None' and combined_dict_gemini['Labels'][i] == 'None':
        true_neg.append(i)
    elif predicted[i]['Label'] != 'None' and combined_dict_gemini['Labels'][i] != 'None':
        true_pos.append(i)
    elif predicted[i]['Label'] != 'None' and combined_dict_gemini['Labels'][i] == 'None':
        false_pos.append(i)
    elif predicted[i]['Label'] == 'None' and combined_dict_gemini['Labels'][i] != 'None':
        false_neg.append(i)
    n += 1
    
tp = len(true_pos)
tn = len(true_neg)
fp = len(false_pos)
fn = len(false_neg)
predicted_no = tn+fn
predicted_yes = tp+fp
actual_no = tn+fp
actual_yes = fn+tp
print(n,tn,tp,fp,fn)
accuracy = (tn+tp)/n
misclassification_rate = (fp+fn)/n # aka error rate
# assert 1-accuracy == misclassification_rate
true_pos_rate = tp / actual_yes # aka sensitivity or recall
false_pos_rate = 0 if actual_no == 0 else fp / actual_no 
true_neg_rate = 0 if actual_no == 0 else tn / actual_no
# assert 1-true_neg_rate == false_pos_rate
precision = tp / predicted_yes
prevalence = actual_yes / n
null_error_rate = actual_no / n
f1 = (2*tp) / (2*tp + fp + fn) # harmonic mean of precision and recall

1396 998 349 1 48


In [44]:
gemini_stats = {
    "tp": tp,
    "tn": tn,
    "fp": fp,
    "fn": fn,
    "predicted_no": predicted_no,
    "predicted_yes": predicted_yes,
    "actual_no": actual_no,
    "actual_yes": actual_yes,
    "accuracy": accuracy,
    "misclassification_rate": misclassification_rate,
    "true_pos_rate": true_pos_rate,
    "false_pos_rate": false_pos_rate,
    "true_neg_rate": true_neg_rate,
    "precision": precision,
    "prevalence": prevalence,
    "null_error_rate": null_error_rate,
    "f1": f1
}
print("Gemini Stats:")
print(gemini_stats)

Gemini Stats:
{'tp': 349, 'tn': 998, 'fp': 1, 'fn': 48, 'predicted_no': 1046, 'predicted_yes': 350, 'actual_no': 999, 'actual_yes': 397, 'accuracy': 0.9648997134670487, 'misclassification_rate': 0.03510028653295129, 'true_pos_rate': 0.8790931989924433, 'false_pos_rate': 0.001001001001001001, 'true_neg_rate': 0.998998998998999, 'precision': 0.9971428571428571, 'prevalence': 0.2843839541547278, 'null_error_rate': 0.7156160458452722, 'f1': 0.9344042838018741}


In [45]:
json_file_stats_gemini = json.dumps({"gemini_stats": gemini_stats}, indent=4)

with open("stats_gemini.json", "w") as file:
    file.write(json_file_stats_gemini)

In [46]:
n = 0 # number of tweets that were able to be classified without being blocked by safety filters - 
confusion = {'actual_propaganda':{'predicted_propaganda': 0,'predicted_radicalization': 0, 'predicted_recruitment': 0},'actual_radicalization':{'predicted_propaganda': 0,'predicted_radicalization': 0, 'predicted_recruitment': 0},'actual_recruitment':{'predicted_propaganda': 0,'predicted_radicalization': 0, 'predicted_recruitment': 0}}
for i in range(len(combined_dict_gemini['Labels'])):
    if 'Label' not in predicted[i]:
        continue
    if predicted[i]['Label'] == 'Propaganda' and combined_dict_gemini['Labels'][i] == 'Propaganda':
        confusion['actual_propaganda']['predicted_propaganda'] += 1
    elif predicted[i]['Label'] == 'Radicalization' and combined_dict_gemini['Labels'][i] == 'Propaganda':
        confusion['actual_propaganda']['predicted_radicalization'] += 1
    elif predicted[i]['Label'] == 'Recruitment' and combined_dict_gemini['Labels'][i] == 'Propaganda':
        confusion['actual_propaganda']['predicted_recruitment'] += 1
    elif predicted[i]['Label'] == 'Propaganda' and combined_dict_gemini['Labels'][i] == 'Radicalization':
        confusion['actual_radicalization']['predicted_propaganda'] += 1
    elif predicted[i]['Label'] == 'Radicalization' and combined_dict_gemini['Labels'][i] == 'Radicalization':
        confusion['actual_radicalization']['predicted_radicalization'] += 1
    elif predicted[i]['Label'] == 'Recruitment' and combined_dict_gemini['Labels'][i] == 'Radicalization':
        confusion['actual_radicalization']['predicted_recruitment'] += 1
    elif predicted[i]['Label'] == 'Propaganda' and combined_dict_gemini['Labels'][i] == 'Recruitment':
        confusion['actual_recruitment']['predicted_propaganda'] += 1
    elif predicted[i]['Label'] == 'Radicalization' and combined_dict_gemini['Labels'][i] == 'Recruitment':
        confusion['actual_recruitment']['predicted_radicalization'] += 1
    elif predicted[i]['Label'] == 'Recruitment' and combined_dict_gemini['Labels'][i] == 'Recruitment':
        confusion['actual_recruitment']['predicted_recruitment'] += 1
    n += 1
print(confusion)

{'actual_propaganda': {'predicted_propaganda': 154, 'predicted_radicalization': 53, 'predicted_recruitment': 0}, 'actual_radicalization': {'predicted_propaganda': 37, 'predicted_radicalization': 23, 'predicted_recruitment': 2}, 'actual_recruitment': {'predicted_propaganda': 22, 'predicted_radicalization': 27, 'predicted_recruitment': 31}}


In [47]:
json_file_sub_confusion_gemini = json.dumps({"sub_confusion_matrix": confusion}, indent=4)

with open("sub_confusion_gemini.json", "w") as file:
    file.write(json_file_sub_confusion_gemini)